In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 98.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 28.6 MB/s eta 0:00:00


In [1]:
import os
import torch
from transformers import BertTokenizerFast, BertConfig, BertForMaskedLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
class LargeTextDataset(torch.utils.data.IterableDataset):
    def __init__(self, file_path, tokenizer, block_size, stride):
        self.file_path = file_path
        self.tokenizer = tokenizer
        self.block_size = block_size
        self.stride = stride

    def __iter__(self):
        with open(self.file_path, "r", encoding="utf-8") as file:
            text_buffer = ""
            for line in file:
                text_buffer += line
                while len(text_buffer) > self.block_size:
                    tokens = self.tokenizer(
                        text_buffer[:self.block_size],
                        padding="max_length",
                        truncation=True,
                        max_length=self.block_size,
                        return_tensors="pt"
                    )
                    text_buffer = text_buffer[self.stride:]
                    input_ids = tokens['input_ids'].squeeze(0)
                    attention_mask = tokens['attention_mask'].squeeze(0)
                    yield {"input_ids": input_ids, "attention_mask": attention_mask}


In [5]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

file_path = "/content/drive/MyDrive/Coursework/large_file.txt"
block_size = 512
stride = 512
dataset = LargeTextDataset(file_path, tokenizer, block_size, stride)


In [4]:
config = BertConfig(
    vocab_size=tokenizer.vocab_size,
    hidden_size=768,
    num_hidden_layers=12,
    num_attention_heads=12,
    intermediate_size=3072,
    hidden_act="gelu",
    hidden_dropout_prob=0.1,
    attention_probs_dropout_prob=0.1,
    max_position_embeddings=512,
    type_vocab_size=1,
    initializer_range=0.02,
    layer_norm_eps=1e-12,
    gradient_checkpointing=False,
    position_embedding_type="absolute",
    use_cache=True,
)

model = BertForMaskedLM(config)


NameError: name 'tokenizer' is not defined

In [7]:
file_size = os.path.getsize(file_path)
approx_tokens_per_block = block_size // stride
approx_total_tokens = file_size * approx_tokens_per_block
approx_steps = approx_total_tokens // (512) 

training_args = TrainingArguments(
    output_dir="output",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    max_steps=approx_steps
)


In [8]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,3.814400
1000,3.324800
1500,3.316300
2000,3.310900
2500,3.317000
3000,3.308400
3500,3.310000
4000,3.301800
4500,3.300100
5000,3.297300


In [ ]:
trainer.save_model("output")